In [13]:
# Boggle: finding all valid words listed in an dictionary from an charachter matrix
#
# rules: The aim of the game is to find as many words as possible in a 4-by-4 grid randomly 
# filled with letters. You are allowed to go up, down, left, right, or diagonally, but not 
# use the same letter more than once.
#
# key hints: two subproblems
# 1. graph for traverse the matrix using DFS
# 2. a TRIE or prefix tree to track all valid words in a dictionary
#
# prefix tree class
#
#               ''
#                +
#    +---+-------+-------+-------+
#    A   B       C       D       T
#    +   +       +       +     +   +
#    N   Y       A       O     A   U
#    |   |    /  |  \    |     |   |
#    +   +   +   +   +   +     +   +
#    T   T   T   N   R   G     N   B
#                              +   +
#                              K   E
#
class PrefixTree(object):
    def __init__(self, letter=None):
        self.letter = letter
        self.children = {}
        self.stop = False
 
    def add(self, word):
        if len(word):
            letter = word[0]
            word = word[1:]
            if letter not in self.children:
                self.children[letter] = PrefixTree(letter);
            return self.children[letter].add(word)
        else:
            self.stop = True
            return self
 
    def find_letter(self, letter):
        if letter not in self.children:
            return None
        return self.children[letter]

    def print_tree(self, level):
        for i in range(level):
            print(" ", end = '')
        print(self.letter, end = '')
        if self.stop:
            print("\n")
        for letter in self.children:
            child=self.children[letter]
            child.print_tree(level+1)
            
    def __repr__(self):
        return "PrefixTree(letter={0}, stop={1})".format(self.letter, self.stop)

# Here’s the code for the game solver itself. It simply searches recursively starting at each cell in the game using DFS 
# and looks up the sequences of letters it builds up in the prefix tree:
from random import choice

class Boggle(object):
    def __init__(self, board=None):
        self.size = 4
        if board is None:
            self.board = []
            for i in range(0, self.size):
                self.board.append([])
                for j in range(0, self.size):
                    self.board[i].append(Boggle.random_letter())
        else:
            self.board = board
 
    @staticmethod
    def random_letter():
        return chr(choice(range(65,91)))
 
    def play(self, tree, found):
        visited = []
        for r in range(0, self.size):
            for c in range(0, self.size):
                # avoid revisit the starting cell with same latter visited before
                # to reduce redundant search
                if self.board[r][c] not in visited:
                    visited.append(self.board[r][c])
                    self.search_r(tree, found, r, c)
 
    def search_r(self, tree, found, row, col, path=None, node=None, word=None):
        letter = self.board[row][col]
        if node is None or path is None or word is None:
            node = tree.find_letter(letter)
            path = [(row, col)]
            word = letter
        else:
            node = node.find_letter(letter)
            path.append((row, col))
            word = word + letter
        if node is None:
            return
        elif node.stop:
            found.add(word)
        for r in range(row - 1, row + 2):
            for c in range(col - 1, col + 2):
                if (r >= 0 and r < self.size
                        and c >= 0 and c < self.size 
                        and not (r == row and c == col) 
                        and (r, c) not in path):
                    self.search_r(tree, found, r, c, path[:], node, word[:])
 
    def __repr__(self):
        return "Boggle(size={0}, board={1})".format(self.size, self.board)
    
# expected output: ANT, CAN, CAT, TANK, TUBE
def main():
    dictionary = ['CAT','DOG','BYTE','TUBE', 'CAN','ANT','CAR','TANK']
    prefixTree = PrefixTree('')
    for word in dictionary:
        prefixTree.add(word)
    prefixTree.print_tree(0)
    
    board =[
        ['C','J','Z','E'],
        ['V','A','X','B'],
        ['X','N','T','U'],
        ['I','A','N','K']]

    boggle = Boggle(board)
    found = set()
    boggle.play(prefixTree, found)
    for word in sorted(found):
        print(word)
    print(boggle)
 
if __name__ == '__main__':
    main()
    
    




 C  A   T

   N

   R

 D  O   G

 B  Y   T    E

 T  U   B    E

  A   N    K

 A  N   T

ANT
CAN
CAT
TANK
TUBE
Boggle(size=4, board=[['C', 'J', 'Z', 'E'], ['V', 'A', 'X', 'B'], ['X', 'N', 'T', 'U'], ['I', 'A', 'N', 'K']])
